# 3.3 线性回归的简洁实现

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.11.0


## 3.3.1 生成数据集

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据

In [3]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [62]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[ 0.8156, -0.1750],
        [ 0.8432,  0.7630],
        [ 1.3495, -0.8787],
        [-0.2678, -0.1668],
        [-1.3621,  1.3171],
        [-0.1808, -0.5483],
        [ 1.1959,  0.4266],
        [-1.5140, -0.8984],
        [-0.9047, -0.9150],
        [-0.6514,  1.3578]]) 
 tensor([ 6.4282,  3.2906,  9.8940,  4.2145, -2.9920,  5.7025,  5.1366,  4.2373,
         5.5124, -1.7017])


## 3.3.3 定义模型

In [63]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear1', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])


Sequential(
  (linear1): Linear(in_features=2, out_features=1, bias=True)
  (linear2): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


### 查看网络参数
`net.weight、net.bias`参数的修改和使用

`net.parameters()`函数

In [9]:
# 修改weight和bias
net[0].weight.data.normal_(0, 0.01) 
net[0].bias.data.fill_(0)

for param in net.parameters():
    print(param)

print(net[0].weight)
print(net[0].bias)

Parameter containing:
tensor([[0.0054, 0.0066]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)
Parameter containing:
tensor([[ 0.6209, -0.1178]], requires_grad=True)
Parameter containing:
tensor([0.3026], requires_grad=True)
Parameter containing:
tensor([[0.0054, 0.0066]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.4 初始化模型参数

In [66]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [10]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.0054, 0.0066]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)
Parameter containing:
tensor([[ 0.6209, -0.1178]], requires_grad=True)
Parameter containing:
tensor([0.3026], requires_grad=True)


## 3.3.5 定义损失函数

In [13]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [14]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [ ]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [15]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.003
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## 3.3.7 训练模型

In [16]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x1 and 2x1)

In [76]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0007, -3.4010]])
4.2 tensor([4.1999])
